In [1]:
using CSV
using DataFrames
using Dates

In [2]:
ENV["COLUMNS"] = 1000;

In [3]:
rawdata = DataFrame(CSV.File("../rawdata/output_HHS_US_2021-01-16_one_column.csv"))

,Column1,hospital_name,collection_week,fips_code,hospitalization_percentage,hospitalization_2020-12-22,target_date
,Int64,String,Date,Float64,Float64,Float64,Date
1,10,PRATTVILLE BAPTIST HOSPITAL,2020-12-11,1001.0,1.0,3.69904,2020-12-22
2,11,THOMAS HOSPITAL,2020-12-11,1003.0,0.568556,5.64175,2020-12-22
3,12,SOUTH BALDWIN REGIONAL MEDICAL CENTER,2020-12-11,1003.0,0.305302,3.02949,2020-12-22
4,13,NORTH BALDWIN INFIRMARY,2020-12-11,1003.0,0.126143,1.25171,2020-12-22
5,14,MEDICAL CENTER BARBOUR,2020-12-11,1005.0,1.0,0.640395,2020-12-22
6,15,BIBB MEDICAL CENTER,2020-12-11,1007.0,0.0,0.0,2020-12-22
7,16,ST VINCENTS BLOUNT,2020-12-11,1009.0,1.0,4.11525,2020-12-22
8,17,BULLOCK COUNTY HOSPITAL,2020-12-11,1011.0,0.0,0.0,2020-12-22
9,18,REGIONAL MEDICAL CENTER OF CENTRAL ALABAMA,2020-12-11,1013.0,1.0,0.746617,2020-12-22


In [4]:
data = select(rawdata,
    :hospital_name => ByRow(titlecase) => :hospital,
    :target_date => :date,
    "hospitalization_2020-12-22" => ByRow(x -> x*0.3) => :admitted_moderate_icu,
    "hospitalization_2020-12-22" => ByRow(x -> x*0.7) => :admitted_moderate_acute,
    "hospitalization_2020-12-22" => :admitted_moderate_allbeds,
)

,hospital,date,admitted_moderate_icu,admitted_moderate_acute,admitted_moderate_allbeds
,String,Date,Float64,Float64,Float64
1,Prattville Baptist Hospital,2020-12-22,1.10971,2.58933,3.69904
2,Thomas Hospital,2020-12-22,1.69252,3.94922,5.64175
3,South Baldwin Regional Medical Center,2020-12-22,0.908847,2.12064,3.02949
4,North Baldwin Infirmary,2020-12-22,0.375512,0.876194,1.25171
5,Medical Center Barbour,2020-12-22,0.192119,0.448277,0.640395
6,Bibb Medical Center,2020-12-22,0.0,0.0,0.0
7,St Vincents Blount,2020-12-22,1.23457,2.88067,4.11525
8,Bullock County Hospital,2020-12-22,0.0,0.0,0.0
9,Regional Medical Center Of Central Alabama,2020-12-22,0.223985,0.522632,0.746617


In [5]:
data |> CSV.write("../data/hhs_forecast_2020_12_26.csv")

"../data/hhs_forecast_2020_12_26.csv"